RAC

In [ ]:
!pip install -qU langchain langchain-openai chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.2 MB/s eta 0:00

In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
pip install pinecone-client langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.14
    Uninstalling aiohttp-3.11.14:
      Successfully uninstalled aiohttp-3.11.14
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.45
    Uninstalling langchain-core-0.3.45:
  

In [15]:
import os
import getpass
import pinecone
import bs4
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Obtener claves API desde el usuario si no están definidas
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

if "PINECONE_API_KEY" not in os.environ:
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

if "PINECONE_ENV" not in os.environ:
    os.environ["PINECONE_ENV"] = input("Enter your Pinecone environment: ")

index_name = "rac1"

# Inicializar Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# Verificar si el índice ya existe, si no, crearlo
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # Dimensión para OpenAI embeddings
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=os.environ["PINECONE_ENV"])
    )

# Cargar contenido desde Wikipedia
loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/World_War_II",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(["p", "h1", "h2"])
    ),
)
documents = loader.load()

# Dividir el contenido en fragmentos
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents)

# Generar embeddings con OpenAI
embedding_function = OpenAIEmbeddings()

# Conectar con Pinecone y guardar los embeddings
vector_db = PineconeVectorStore.from_documents(all_splits, embedding_function, index_name=index_name)

# Crear el retriever
retriever = vector_db.as_retriever()

# Inicializar modelo de lenguaje
llm = ChatOpenAI(model_name="gpt-4o-mini")

# Crear pipeline de RAG con retorno de documentos fuente
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, return_source_documents=True)

# Hacer una pregunta al sistema
query = "Explain to me the war world 2"
result = qa_chain(query)

# Mostrar respuesta y documentos fuente
print("Respuesta:", result["result"])
print("Documentos fuente:", result["source_documents"])


Respuesta: World War II, also known as the Second World War, was a global conflict that lasted from September 1, 1939, to September 2, 1945. It involved two main coalitions: the Allies and the Axis powers. Nearly all of the world's countries participated in the war, which mobilized vast resources in pursuit of total war.

The war was characterized by significant military innovations, including the use of tanks and aircraft, which played major roles in battles and enabled strategic bombing of cities. Notably, it was during this conflict that the first and only nuclear weapons were used in warfare.

World War II was the deadliest conflict in history, resulting in an estimated 70 to 85 million deaths, with more than half of the casualties being civilians. Many people died as a result of genocides, including the Holocaust, as well as from massacres, starvation, and disease.

After the Allied victory, several countries, including Germany, Austria, Japan, and Korea, were occupied, and leader